In [ ]:
# 실패 로그 기반 데이터 재수집
import os
import pandas as pd
import requests
import time
from datetime import datetime
from tqdm import tqdm
import ssl
import warnings
import xml.etree.ElementTree as ET
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from dotenv import load_dotenv

# 환경 및 경고 설정
load_dotenv()
warnings.filterwarnings('ignore', category=InsecureRequestWarning)
ssl._create_default_https_context = ssl._create_unverified_context

# 상수
API_KEY = os.getenv("DO_API_KEY")
BASE_URL = 'http://apis.data.go.kr/B552845/katSale/trades'
ITEM_CODES = {"상추": "1105"}
max_retries = 2  # 재시도 최대 횟수 (1회 시도 + 0회 재시도)

# 디렉토리 준비
os.makedirs("logs", exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("success", exist_ok=True)

# 도매시장 코드 불러오기
df_market = pd.read_csv("도매시장_코드.csv", encoding="cp949", header=None)
df_market[0] = df_market[0].astype(str)

# 실패 로그 불러오기
fail_df = pd.read_csv("유통공사_fail_log.csv", encoding="utf-8")
fail_pairs = fail_df[['mcode', 'date']].drop_duplicates()
fail_pairs['mcode'] = fail_pairs['mcode'].astype(str)

for item_name, code in ITEM_CODES.items():
    LARGE = code[:2]
    MID = code[2:]
    data_list = []
    cnt =0

    print(f"\n📦 실패 항목 재시도 시작: {item_name}")
    for _, row in tqdm(fail_pairs.iterrows(), total=len(fail_pairs), desc="재시도 진행"):
        mcode = str(row['mcode'])
        date_str = row['date']

        market_name_row = df_market[df_market[0] == mcode]
        if market_name_row.empty:
            print(f"❌ 시장 코드 {mcode} 누락 - 스킵")
            continue
        market_name = market_name_row.values[0][1]

        retry_count = 0
        market_success = False


        while retry_count < max_retries:
            page_no = 1
            cnt += 1
            try:
                while True:
                    print(f"▶️ 요청 시도: {item_name} | 시장코드: {mcode} | 날짜: {date_str} | 페이지: {page_no} | 재시도: {retry_count + 1}")

                    params = {
                        'serviceKey': API_KEY,
                        'pageNo': page_no,
                        'numOfRows': 100,
                        'cond[trd_clcln_ymd::EQ]': date_str,
                        'cond[whsl_mrkt_cd::EQ]': mcode,
                        'cond[gds_lclsf_cd::EQ]': LARGE,
                        'cond[gds_mclsf_cd::EQ]': MID
                    }

                    response = requests.get(BASE_URL, params=params, verify=False, timeout=10)
                    content_type = response.headers.get("Content-Type", "")
                    time.sleep(1.0)
                    response_preview = response.text[:500].strip()

                    # 에러 체크
                    if "LIMITED_" in response_preview:
                        fail_reason = "❌ API 호출 제한 (LIMITED_ 응답)"
                    elif "SERVICE ERROR" in response_preview:
                        fail_reason = "❌ 서비스 오류 (SERVICE ERROR 응답)"
                    elif "ERROR" in response_preview.upper():
                        fail_reason = "❌ 기타 오류 포함 (ERROR 키워드 포함)"
                    elif "TOO MANY REQUESTS" in response_preview.upper():
                        fail_reason = "❌ 요청 과다로 인한 제한 (Too Many Requests)"
                    else:
                        fail_reason = None

                    if fail_reason:
                        print(f"⛔ {fail_reason} - 재시도 대기 중 (2분)")
                        log_prefix = f"logs/retry_failed_{item_name}_{mcode}_{date_str}"
                        with open(f"{log_prefix}.html", "w", encoding="utf-8") as f:
                            f.write(response.text)
                        with open(f"{log_prefix}_info.txt", "w", encoding="utf-8") as f:
                            f.write(f"[오류] {fail_reason}\n{response_preview}")
                        retry_count += 1
                        if retry_count >= max_retries:
                            print(f"❗ 최대 재시도 {max_retries}회 초과 - 중단")
                            break
                        time.sleep(60)
                        continue

                    # 응답 파싱
                    if "application/json" in content_type:
                        json_data = response.json()
                        body = json_data.get("response", {}).get("body", {})
                        items = body.get("items", {}).get("item", [])
                        total_count = int(body.get("totalCount", 0))

                    elif "application/xml" in content_type or response.text.strip().startswith("<"):
                        root = ET.fromstring(response.text)
                        total_count_el = root.find(".//totalCount")
                        total_count = int(total_count_el.text) if total_count_el is not None else 0
                        item_els = root.findall(".//item")
                        items = [{el.tag: el.text for el in item} for item in item_els]

                    else:
                        raise ValueError(f"알 수 없는 응답 형식: {content_type}")

                    if not items:
                        print("⚠️ 거래 데이터 없음")
                        market_success = True
                        break

                    data_list.extend(items)

                    if cnt%10000==0 :
                        print(f"🧪 중간 저장 시도: 현재 data_list 길이 = {len(data_list)}")
                        mid_save_path = f"data/retry/유통공사_retry_{item_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}_mid.csv"
                        df_mid = pd.DataFrame(data_list)
                        df_mid.to_csv(mid_save_path, encoding='cp949', index=False)
                        print(f"💾 중간 저장 완료: {mid_save_path}")
                        time.sleep(0.1)

                    market_success = True
                    if page_no * 100 >= total_count:
                        print(f"✅ 마지막 페이지 도달 (totalCount: {total_count})")
                        break
                    if page_no > 10:
                        print("🚨 페이지 10 초과 - 무한 루프 방지를 위해 중단")
                        break

                    page_no += 1
                    time.sleep(1.0)

                if market_success:
                    break
                else:
                    retry_count += 1
                    if retry_count >= max_retries:
                        print(f"❗ 최대 재시도 {max_retries}회 초과 - 중단")
                        break
                    time.sleep(2 * retry_count)

            except Exception as e:
                retry_count += 1
                print(f"❗예외 발생: {e} (재시도 {retry_count}/{max_retries})")
                fail_log_prefix = f"logs/retry_failed_{item_name}_{mcode}_{date_str}_try{retry_count}"
                if 'response' in locals():
                    with open(f"{fail_log_prefix}.txt", "w", encoding="utf-8") as f:
                        f.write(response.text)
                with open(f"{fail_log_prefix}_info.txt", "w", encoding="utf-8") as f:
                    f.write(f"[예외] {str(e)}\n")
                if retry_count >= max_retries:
                    print(f"❗ 최대 재시도 {max_retries}회 초과 - 중단")
                    break
                time.sleep(2 * retry_count)

        if not market_success:
            fail_log_path = f"data/logs/retry_failed_{item_name}_{mcode}_{date_str}.txt"
            with open(fail_log_path, "w", encoding="utf-8") as f:
                f.write(f"❌ {datetime.now()} - {item_name} {mcode} {date_str} 데이터 수집 실패\n")

    # DataFrame 생성 전 타입 검사
    if data_list:
        if not all(isinstance(item, dict) for item in data_list):
            raise ValueError("data_list에는 dict가 아닌 항목이 있습니다.")

        df = pd.DataFrame(data_list)
        filename = f"data/유통공사_retry_{item_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, encoding='cp949', index=False)
        print(f"✅ 저장 완료: {filename}")
    else:
        print(f"⚠️ {item_name}: 재시도에서도 데이터 없음")




📦 실패 항목 재시도 시작: 양파



재시도 진행:   0%|                                                                           | 0/60374 [00:00<?, ?it/s]

▶️ 요청 시도: 양파 | 시장코드: 310401 | 날짜: 2018-02-04 | 페이지: 1 | 재시도: 1



재시도 진행:   0%|                                                                | 1/60374 [00:01<18:50:46,  1.12s/it]

⚠️ 거래 데이터 없음
▶️ 요청 시도: 양파 | 시장코드: 310901 | 날짜: 2018-02-04 | 페이지: 1 | 재시도: 1


...

⚠️ 거래 데이터 없음
▶️ 요청 시도: 양파 | 시장코드: 110001 | 날짜: 2019-01-27 | 페이지: 1 | 재시도: 1



재시도 진행:   6%|███▎                                                       | 3356/60374 [1:04:17<17:33:15,  1.11s/it]

⚠️ 거래 데이터 없음
▶️ 요청 시도: 양파 | 시장코드: 210005 | 날짜: 2019-01-28 | 페이지: 1 | 재시도: 1



재시도 진행:   6%|███▎                                                       | 3357/60374 [1:04:18<17:35:44,  1.11s/it]

⚠️ 거래 데이터 없음
▶️ 요청 시도: 양파 | 시장코드: 370401 | 날짜: 2019-01-28 | 페이지: 1 | 재시도: 1


In [3]:
import os

print(os.getcwd())


C:\ai_x\source\JikFam
